In [1]:
#pip install gradio
#pip install --upgrade gradio

In [2]:
import gradio as gr
import cv2
import numpy as np

In [3]:
from tensorflow.keras.models import load_model

# Load the previously saved model
model1 = load_model('Plant_Seedling_Model1.h5')

In [4]:
def image_preprocessing(images):

    new_train = []
    sets = []; 
    #getEx = True
    for i in images:
        blurr = cv2.GaussianBlur(i,(5,5),0)
        hsv = cv2.cvtColor(blurr,cv2.COLOR_BGR2HSV) #Using BGR TO HSV conversion. reason is mentioned above
        #HSV Boundaries for the Green color (GREEN PARAMETERS)
        lower = (25,40,50)
        upper = (75,255,255)
        mask = cv2.inRange(hsv,lower,upper) # create a mask 
        struc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)) #getting structring element ( kernal) of sie 11x11
        mask = cv2.morphologyEx(mask,cv2.MORPH_CLOSE,struc) # applying morphological transformation ( closing operation ) to remove imperfections from the binary image ( mask)
        boolean = mask>0
        new = np.zeros_like(i,np.uint8)
        new[boolean] = i[boolean]
        new_train.append(new)

    new_train = np.asarray(new_train)
    new_train  = new_train / 255

    return new_train


In [7]:
def predict_plant_seedling(img):
    # Wrap the image in a list, preprocess, and then extract it back
    #img = img[..., ::-1]
    resized_image = cv2.resize(img, (128, 128))
    #img = cv2.cvtColor(resized_image, cv2.COLOR_RGB2BGR)
    
    processed_images = image_preprocessing([resized_image ])
    
    processed_image = processed_images[0]  # Extract the single processed image

    # Add batch dimension
    processed_image = np.expand_dims(processed_image, axis=0)

    # Predict using the trained model
    prediction = model1.predict(processed_image)
    predicted_class = np.argmax(prediction, axis=1)
    print(f"Class {predicted_class}")
    
    # Map to class names
    class_names = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common Wheat', 'Fat Hen', 'Loose Silky-bent', 'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']
    predicted_class_name = class_names[predicted_class[0]]

    return predicted_class_name


In [8]:
iface = gr.Interface(fn=predict_plant_seedling, inputs="image", outputs="text")
iface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 113ms/step
Class [11]
1/1 [==============================] - 0s 20ms/step
Class [5]
1/1 [==============================] - 0s 19ms/step
Class [0]
1/1 [==============================] - 0s 20ms/step
Class [0]
